# DATA CLEANING

### Objectives

* Data description.
* Determine if data transformation is needed.
* Check the dataset for missing data and placeholders.
* Check dataset for duplicates.
* Creating new columns via feature engineering.
* Select subet data.
* Save the new dataset.

## IMPORT DATA

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('/CSV_Files/thermostat_data.csv')
data.head()

,building_id,ts,outdoor_temp,boiler_on,commercial,state,indoor_temp
0,6168,1605644640,45.607,1.0,False,NY,NaN
1,6168,1605644700,45.670,1.0,False,NY,NaN
2,6168,1605644760,45.531,1.0,False,NY,NaN
3,6168,1605644820,45.353,1.0,False,NY,NaN
4,6168,1605644880,45.137,1.0,False,NY,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3608749 entries, 0 to 3608748
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   building_id   int64  
 1   ts            int64  
 2   outdoor_temp  float64
 3   boiler_on     float64
 4   commercial    bool   
 5   state         object 
 6   indoor_temp   float64
dtypes: bool(1), float64(3), int64(2), object(1)
memory usage: 168.6+ MB


## DATA DESCRIPTION

The data is consist of 7 columns and 3,608,749 rows, where each row contains the measurements for one minute from	*2020-11-17 20:24:00* to *2020-12-17 20:22:00*.

*Within the data, following columns are provided.*


*   **building_id  :** Numerical type data, used as unique identifier for each building. There are 89 different buildings in the dataset.

*   **ts :** Unix timestamp. 

*   **outdoor_temp :** Outdoor temperature of the city in Fahrenheit that the building in question is in

*   **boiler_on    :** Binary column that contains boiler on/off information.

*   **commercial   :** Boolean column that answers if the building is commercial or not.

*   **state        :** In which state the building in question is. Company has active systems in NY, MA, and RI.

*   **indoor_temp  :** Indoor temperature of the building in Fahrenheit.

In [5]:
 # Check each column for unique values.
for col in data.columns:
    print(col, '\n\n', data[col].value_counts())
    print(40*'-')

building_id 

 4699461536766    42127
4753626976665    42024
8250             42000
4621758970989    41998
4613394659241    41995
                 ...  
4599460339872    31846
4810168312158    22985
4812181018245    22862
4822071968685    14448
6492             11413
Name: building_id, Length: 89, dtype: int64
----------------------------------------
ts 

 1607330520    174
1606258680    171
1606264140    170
1606063080    170
1605996120    170
             ... 
1606345740      1
1606316640      1
1606383420      1
1606367700      1
1606355460      1
Name: ts, Length: 42870, dtype: int64
----------------------------------------
outdoor_temp 

 -79.158     2305
-78.642     2252
-77.643     2149
-77.159     2059
-78.137     2047
            ... 
 18.649        1
 119.039       1
 119.539       1
-22.390        1
 1.040         1
Name: outdoor_temp, Length: 8199, dtype: int64
----------------------------------------
boiler_on 

 0.0    2275825
1.0     825948
Name: boiler_on, dtype: int64


## DATA TRANSFORMATION

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3608749 entries, 0 to 3608748
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   building_id   int64  
 1   ts            int64  
 2   outdoor_temp  float64
 3   boiler_on     float64
 4   commercial    bool   
 5   state         object 
 6   indoor_temp   float64
dtypes: bool(1), float64(3), int64(2), object(1)
memory usage: 168.6+ MB


**building_id** column is a unique identifier for each building. This is why we save this column as an *object* data type.

In [7]:
data['building_id'] = data['building_id'].astype('object')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3608749 entries, 0 to 3608748
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   building_id   object 
 1   ts            int64  
 2   outdoor_temp  float64
 3   boiler_on     float64
 4   commercial    bool   
 5   state         object 
 6   indoor_temp   float64
dtypes: bool(1), float64(3), int64(1), object(2)
memory usage: 168.6+ MB


**ts** column is unix timestampwhich we will convert to pandas *datetime* object.

In [10]:
# Unix timestamp can be stored in smaller spaces. 
data['ts'] = pd.to_datetime(data['ts'], unit='s')

**commercial** is a boolean type column which will be transformed into binary column.

In [11]:
data['commercial'].unique()

array([False,  True])

In [12]:
data['commercial'] = data['commercial'].map({False : 0, True : 1})
data['commercial'].value_counts()

0    3482808
1     125941
Name: commercial, dtype: int64

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3608749 entries, 0 to 3608748
Data columns (total 7 columns):
 #   Column        Dtype         
---  ------        -----         
 0   building_id   object        
 1   ts            datetime64[ns]
 2   outdoor_temp  float64       
 3   boiler_on     float64       
 4   commercial    int64         
 5   state         object        
 6   indoor_temp   float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 192.7+ MB


## MISSING DATA

In [14]:
# Check for null values in the dataset.
data.isnull().sum()

building_id          0
ts                   0
outdoor_temp    122630
boiler_on       506976
commercial           0
state                0
indoor_temp     415159
dtype: int64

Our dataset has null values in 3 columns : *outdoor_temp*, *boiler_on*, *indoor_temp*.

Let's investigate each building seperately.

In [ ]:
for bldg_id in list(data['building_id'].unique()):
    print(bldg_id)
    print('\n', data[data['building_id'] == bldg_id].info())
    print('Sum of nans:', data[data['building_id'] == bldg_id].isnull().sum())
    print(40*'-')
    print('\n\n')

We can simply assume the next temperature is the same as the current temperature. So we use front fill method to fill NaN values.

In [18]:
data.fillna(method='ffill', inplace=True)

In [19]:
data.isnull().sum()

building_id         0
ts                  0
outdoor_temp        0
boiler_on           0
commercial          0
state               0
indoor_temp     83834
dtype: int64

In [20]:
for bldg_id in list(data['building_id'].unique()):
    print(bldg_id)
    print('\n', data[data['building_id'] == bldg_id].info())
    print('Sum of nans:', data[data['building_id'] == bldg_id].isnull().sum())
    print(40*'-')
    print('\n\n')

6168
<class 'pandas.core.frame.DataFrame'>
Int64Index: 41982 entries, 0 to 41981
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   building_id   41982 non-null  int64         
 1   ts            41982 non-null  datetime64[ns]
 2   outdoor_temp  41982 non-null  float64       
 3   boiler_on     41982 non-null  float64       
 4   commercial    41982 non-null  int64         
 5   state         41982 non-null  object        
 6   indoor_temp   0 non-null      float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 2.6+ MB

 None
Sum of nans: building_id         0
ts                  0
outdoor_temp        0
boiler_on           0
commercial          0
state               0
indoor_temp     41982
dtype: int64
----------------------------------------



6204
<class 'pandas.core.frame.DataFrame'>
Int64Index: 41852 entries, 41982 to 83833
Data columns (total 7 columns):
 #   Col

We can observe that buldings with **6168** and **6204** id numbers have no indoor temperature information. We can drop these two buildings from our dataset.

In [21]:
data.dropna(inplace=True)

In [22]:
data.reset_index(inplace=True)
data.drop('index', axis=1, inplace=True)
data.head()

,building_id,ts,outdoor_temp,boiler_on,commercial,state,indoor_temp
0,6294,2020-11-17 20:24:00,46.748,0.0,0,NY,76.6015
1,6294,2020-11-17 20:25:00,46.798,0.0,0,NY,76.6015
2,6294,2020-11-17 20:26:00,46.810,0.0,0,NY,76.6015
3,6294,2020-11-17 20:27:00,46.636,0.0,0,NY,76.6015
4,6294,2020-11-17 20:28:00,46.224,0.0,0,NY,76.6015


## FEATURE ENGINEERING

We create a column to see the temperature difference between the outdoors and indoors.

In [23]:
data['temp_diff'] = data['indoor_temp'] - data['outdoor_temp']
data.head()

,building_id,ts,outdoor_temp,boiler_on,commercial,state,indoor_temp,temp_diff
0,6294,2020-11-17 20:24:00,46.748,0.0,0,NY,76.6015,29.8535
1,6294,2020-11-17 20:25:00,46.798,0.0,0,NY,76.6015,29.8035
2,6294,2020-11-17 20:26:00,46.810,0.0,0,NY,76.6015,29.7915
3,6294,2020-11-17 20:27:00,46.636,0.0,0,NY,76.6015,29.9655
4,6294,2020-11-17 20:28:00,46.224,0.0,0,NY,76.6015,30.3775


## SUBSET DATA

For modeling, we will create a subset data from one building's information for a month. We chose building **4599162238458** as our pilot building for modeling.

In [24]:
df = data[data['building_id'] == 4599162238458]
df.head(10)

,building_id,ts,outdoor_temp,boiler_on,commercial,state,indoor_temp,temp_diff
974855,4599162238458,2020-11-17 20:24:00,51.998,1.0,0,NY,73.7375,21.7395
974856,4599162238458,2020-11-17 20:25:00,51.895,1.0,0,NY,73.7375,21.8425
974857,4599162238458,2020-11-17 20:26:00,51.792,1.0,0,NY,73.7375,21.9455
974858,4599162238458,2020-11-17 20:27:00,51.331,1.0,0,NY,73.7375,22.4065
974859,4599162238458,2020-11-17 20:28:00,51.343,1.0,0,NY,73.7375,22.3945
974860,4599162238458,2020-11-17 20:29:00,51.100,1.0,0,NY,73.7375,22.6375
974861,4599162238458,2020-11-17 20:30:00,50.973,1.0,0,NY,73.7375,22.7645
974862,4599162238458,2020-11-17 20:31:00,50.131,1.0,0,NY,73.7375,23.6065
974863,4599162238458,2020-11-17 20:32:00,49.825,1.0,0,NY,73.7375,23.9125
974864,4599162238458,2020-11-17 20:33:00,50.060,1.0,0,NY,73.5685,23.5085


In [25]:
df.isnull().sum()

building_id     0
ts              0
outdoor_temp    0
boiler_on       0
commercial      0
state           0
indoor_temp     0
temp_diff       0
dtype: int64

## SAVE FINAL DATASET

In [ ]:
data.to_csv('/Users/ulku/Data_Science_2_1/Module_5/CSV_Files/thermostat_data_cleaned.csv')